In [80]:
import numpy as np
import ioh

In [81]:
ioh.ProblemClass.GRAPH.problems

{2000: 'MaxCut2000',
 2001: 'MaxCut2001',
 2002: 'MaxCut2002',
 2003: 'MaxCut2003',
 2004: 'MaxCut2004',
 2100: 'MaxCoverage2100',
 2101: 'MaxCoverage2101',
 2102: 'MaxCoverage2102',
 2103: 'MaxCoverage2103',
 2104: 'MaxCoverage2104',
 2105: 'MaxCoverage2105',
 2106: 'MaxCoverage2106',
 2107: 'MaxCoverage2107',
 2108: 'MaxCoverage2108',
 2109: 'MaxCoverage2109',
 2110: 'MaxCoverage2110',
 2111: 'MaxCoverage2111',
 2112: 'MaxCoverage2112',
 2113: 'MaxCoverage2113',
 2114: 'MaxCoverage2114',
 2115: 'MaxCoverage2115',
 2116: 'MaxCoverage2116',
 2117: 'MaxCoverage2117',
 2118: 'MaxCoverage2118',
 2119: 'MaxCoverage2119',
 2120: 'MaxCoverage2120',
 2121: 'MaxCoverage2121',
 2122: 'MaxCoverage2122',
 2123: 'MaxCoverage2123',
 2124: 'MaxCoverage2124',
 2125: 'MaxCoverage2125',
 2126: 'MaxCoverage2126',
 2127: 'MaxCoverage2127',
 2200: 'MaxInfluence2200',
 2201: 'MaxInfluence2201',
 2202: 'MaxInfluence2202',
 2203: 'MaxInfluence2203',
 2204: 'MaxInfluence2204',
 2205: 'MaxInfluence2205',
 2206

In [267]:
max_cut = ioh.get_problem(2003, problem_class=ioh.ProblemClass.GRAPH)
max_coverage = ioh.get_problem(2101, problem_class=ioh.ProblemClass.GRAPH)

In [83]:
print(max_cut.meta_data)
print(max_coverage.meta_data)

<MetaData: MaxCut2000 id: 2000 iid: 1 dim: 800>
<MetaData: MaxCoverage2100 id: 2100 iid: 1 dim: 450>


In [137]:
nb_it = 10000

In [117]:
x_mcov = cut = np.random.randint(0,2,max_coverage.meta_data.n_variables)
val = max_coverage(x_mcov)
# help(max_coverage.optimum)
for ind in ioh.ProblemClass.GRAPH.problems:
    p = ioh.get_problem(ind, problem_class=ioh.ProblemClass.GRAPH)
    if np.any(p.optimum.x):
        print(p.meta_data)

# 1/2-approximation for MaxCut

Proof in Bertrand Meyer's Advanced Algorithms lecture

In [85]:
max_cut.reset()

In [98]:
cut = np.random.randint(0,2,max_cut.meta_data.n_variables)
max_cut(cut)
print(max_cut.state)

<State evaluations: 1 optimum_found: false current_best: <Solution x: [0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,

# RLS

In [135]:
import numpy as np

class RandomLocalSearch:
    'Random local search algorithm'
    def __init__(self, n: int):
        self.n: int = n
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        'For each it flip 1 bit chosen uar and keep the best of both'
        
        x = np.random.randint(0,2,problem.meta_data.n_variables)
        val = problem(x)
        for _ in range(self.n):
            index_to_flip = np.random.randint(0,problem.meta_data.n_variables)
            y = x
            y[index_to_flip] = 1 - y[index_to_flip]
            val_y = problem(y)
            if val_y > val:
                x = y
                val = val_y

In [232]:
max_coverage.reset()
r = RandomLocalSearch(nb_it)
r(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained, max_coverage.state.y_unconstrained_best)

10001 -226.0 -183.0


In [198]:
max_cut.reset()
r = RandomLocalSearch(nb_it)
r(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained, max_cut.state.y_unconstrained_best)

10001 9576.0 9781.0


# (1+1) EA 

In [144]:
class EA_1p1:
    '(1+1)-EA'
    def __init__(self, n: int, p : float):
        self.n: int = n
        self.p: float = p
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        x = np.random.randint(0,2,problem.meta_data.n_variables)
        val = problem(x)
        for _ in range(self.n):
            y = x.copy()
            for ind in range(len(x)):
                y[ind] = 1 - y[ind] if np.random.binomial(1,self.p) else y[ind]
            val_y = problem(y)
            if val_y > val:
                x = y
                val = val_y

In [261]:
max_coverage.reset()
ea_11 = EA_1p1(nb_it, 1/max_coverage.meta_data.n_variables)
ea_11(max_coverage)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained, max_cut.state.y_unconstrained_best)

0 -inf -inf


In [170]:
max_cut.reset()
ea_11 = EA_1p1(nb_it, np.log(max_cut.meta_data.n_variables)/max_cut.meta_data.n_variables)
ea_11(max_cut)
print(max_cut.state)

<State evaluations: 10001 optimum_found: false current_best: <Solution x: [0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0

# (1+($\lambda, \lambda$)) EA 

In [92]:
max_cut.reset()

In [146]:
import random

class EA_1plamlam:
    '(1+(lam,lam))-EA'
    def __init__(self, n: int, lam : int, p : float, c : float):
        self.n: int = n
        self.lam: int = lam
        self.p: float = p
        self.c: float = c
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        length_x = problem.meta_data.n_variables
        x = np.random.randint(0,2,length_x)
        val = problem(x)
        for _ in range(self.n):
            L = np.random.binomial(length_x, self.p)
            xp = None
            best_val = None # we allow for worse values than x
            for _ in range(self.lam):
                bits_to_flip = random.sample(range(length_x), L)
                new_xp = x.copy()
                for i in bits_to_flip:
                    new_xp[i] = 1 - new_xp[i] if np.random.binomial(1, self.p) else new_xp[i]
            val_new_xp = problem(new_xp)
            if best_val is None or val_new_xp > best_val:
                xp = new_xp
                best_val = val_new_xp 
            y = x.copy()
            best_val = None
            for _ in range(self.lam):
                new_y = x.copy()
                for i in range(length_x):
                    if np.random.binomial(1,self.c):
                        new_y[i] = xp[i]
                val_new_y = problem(new_y)
                if best_val is None or val_new_y > best_val:
                    y = new_y
                    best_val = val_new_y
            if best_val > val:
                x = y
                val = best_val

In [270]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
ea_1lamlam = EA_1plamlam(math.ceil(nb_it / (10)), 10, 1/dim_pb, 1/dim_pb)
ea_1lamlam(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained, max_coverage.state.y_unconstrained_best)

11001 -232.0 -232.0


In [172]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
ea_1lamlam = EA_1plamlam(math.ceil(nb_it / (10)), 10, 1/dim_pb, 1/dim_pb)
ea_1lamlam(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained, max_cut.state.y_unconstrained_best)

11001 9602.0 9609.0


# First idea 
as dealing with submodular functions, we may want to select the arguments that make it grow "the most at first" --> as when adding more elements we face diminishing returns
so from a solution x -> we build lam xs turning off L bits of uar and keeping the best -> we then build lam ys by flipping on L' bits of the best xs uniformly at random and keeping the best y --> then we turn on each bit present in y and not in x

problem: solution x keeps growing?
how to stop that?

In [265]:
class OffAndOn:
    def __init__(self, n: int, lam : int, p_off : float, p_on : float, x_init_0 : bool):
        self.n: int = n
        self.p_off: int = p_off
        self.p_on: int = p_on
        self.lam: float = lam
        self.x_init_0: bool = x_init_0
        
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        if self.x_init_0:
            x = np.zeros(problem.meta_data.n_variables, dtype=int)
        else:
            x = np.random.randint(0,2,problem.meta_data.n_variables)
        val = problem(x)
        for _ in range(self.n):
            L = np.random.binomial(problem.meta_data.n_variables, self.p_off)
            xp = None
            best_val = None 
            for _ in range(self.lam):
                # mutation 1 : we turn off L bits 
                bits_to_flip = random.sample(range(problem.meta_data.n_variables), L)
                new_xp = x.copy()
                for i in bits_to_flip:
                    new_xp[i] = 0
            val_new_xp = problem(new_xp)
            if best_val is None or val_new_xp > best_val:
                xp = new_xp
                best_val = val_new_xp 
            L = np.random.binomial(problem.meta_data.n_variables, self.p_on)
            y = xp.copy()
            best_val = None
            for _ in range(self.lam):
                # mutation 2 : we turn on L bits
                new_y = x.copy()
                bits_to_flip = random.sample(range(problem.meta_data.n_variables), L)
                for i in bits_to_flip:
                    new_y[i] = 1
                val_new_y = problem(new_y)
                if best_val is None or val_new_y > best_val:
                    y = new_y
                    best_val = val_new_y
            # mix:
            # if bit in y and not in x then I turn it on 
            mix = x.copy()
            for i in range(problem.meta_data.n_variables):
                if y[i] and not x[i]:
                    mix[i] = 1
            val_new = problem(mix)
            if val_new > val:
                x = mix
                val = best_val
            

In [248]:
import math
max_cut.reset()
dim_pb = max_cut.meta_data.n_variables
lam = 10
offandon = OffAndOn(math.ceil(nb_it / (lam + 1)), lam, lam/dim_pb, 1/dim_pb, False)
offandon(max_cut)
print(max_cut.state.evaluations, max_cut.state.y_unconstrained_best)

10921 10768.0


In [269]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
lam = 10
offandon = OffAndOn(math.ceil(nb_it / (lam + 1)), lam, lam/dim_pb, 1/dim_pb, True)
offandon(max_coverage)
print(max_coverage.state.evaluations, max_coverage.state.y_unconstrained_best)

10921 416.0
